In [1]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
# make first model

def build_base_model(input_shape):
    input_layer = Input(shape=(input_shape,))
    tanh_output = Dense(1, activation='tanh', name='tanh_output')(input_layer)
    
    model = Model(inputs=input_layer, outputs=tanh_output)
    model.summary()
    return model

In [3]:
# load data
# make sure that the first shape is the IMDB training data. 

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtrain.pickle')
X_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_xtest.pickle')
y_train_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytrain.pickle')
y_test_original = open_pickle('../../data/imdb/imdb_original_preprocessed_ytest.pickle')

In [4]:
# Count vectorizer 

from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
cv = CountVectorizer(min_df = 100, token_pattern=token, lowercase=True, binary=True)
X_train = cv.fit_transform(X_train_original)
X_test = cv.transform(X_test_original)

In [70]:
X_test_original[0]

'this was an excellent show. it came on pbs back home in chicago and i remember cindy herron (from envogue) played the teen aged daughter. the show dealt with subject such as sex, peer pressure and puberty. it was about a middle class black family who had a teen aged daughter and son who moved to a middle class neighborhood from oakland or somewhere (i can not remember). i remember several episode but the one i remember most was when their cousin got her period for the first time. i was probably 7-8 when i first watched it and i was able to keep up with the program. this was a great show. i can not remember the name of the guy who played the son on the show, but i always got him confused with kevin hook.'

In [71]:
X_train_original[0]

'silent night, deadly night 5 is the very last of the series, and like part 4, it is unrelated to the first three except by title and the fact that it is a christmas-themed horror flick.except to the oblivious, there is some obvious thing going on here...mickey rooney plays a toymaker named joe petto and his creepy son\'s name is pino. ring a bell, anyone? now, a little boy named derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. even though it said "do not open till christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. inside is a little red ball that sprouts santa arm and a head, and proceed to kill dad. oop, maybe he should have left well-enough alone. of course derek is then traumatized by the incident since he watched it from the stair, but he does not grow up to be some killer santa, he just stops talking.there is a mysterious stranger lurking around, w

In [69]:
X_test[10]

<1x3686 sparse matrix of type '<class 'numpy.int64'>'
	with 224 stored elements in Compressed Sparse Row format>

In [5]:
def load_unigrams(path, X, y):
    word_list = []
    connotation = {}
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            word_list.append(line.strip())
            
    for word in word_list:
        pos_count = 0
        neg_count = 0
        for i, doc in enumerate(X):
            if word in doc.lower():
                if (y[i] == 1):
                    pos_count += 1
                else:
                    neg_count += 1
                    
        if pos_count > neg_count:
            connotation[word] = 1
        else:
            connotation[word] = 0
    
    return word_list, connotation

def generate_appearance(X_train_corpus, X_test_corpus, word_list, connotation):
    y_train_agreement = []
    for i in range(len(X_train_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_train_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_train_agreement.append(doc_agreement)
        
    y_test_agreement = []
    for i in range(len(X_test_corpus)):
        doc_agreement = []
        for word in word_list:
            if word in X_test_corpus[i]:
                if connotation[word] == 1:
                    doc_agreement.append(1)
                else:
                    doc_agreement.append(-1)
            else:
                doc_agreement.append(0)
        y_test_agreement.append(doc_agreement)
        
    return np.array(y_train_agreement), np.array(y_test_agreement)

# 'imdb-unigrams.txt'

In [6]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [7]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [54]:
# build the combined model
# Combined model
human_terms_len = len(word_list)

base_model = build_base_model(X_train.shape[1])

combined_input_layer = Input(shape=(X_train.shape[1],))

# build the hard coded weight for human terms
ht_input_layer = Input(shape=(human_terms_len,))

# split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)
split = Lambda(layer_split)(ht_input_layer)


# get the document prediction
label_layer = base_model(combined_input_layer)

# stack the multiply layer
dense_layer = []
for i in range(human_terms_len):
    dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones')(Multiply()([split[i], label_layer])))

# concat all the result   
# concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)
concat = Lambda(layer_concat, name='concatenate')(dense_layer)


# pass it to sigmoid layer
output_layer = Dense(1, activation='sigmoid')(concat)

combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 3686)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3687      
Total params: 3,687
Trainable params: 3,687
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 83)           0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 3686)         0                                            
_________________________________________

In [53]:
def layer_split(x):
    return tf.split(x,num_or_size_splits=human_terms_len,axis=1)

def layer_concat(x):
    return tf.concat(x, axis=1)

In [55]:
base_model.compile(loss='mse',
                  optimizer='adam',
                  metrics=['acc'])

combined_model.compile(loss='mse',
                      optimizer='adam',
                      metrics=['acc'])

In [56]:
y_train_tanh = y_train_original
y_train_tanh[y_train_tanh == 0] = -1

In [57]:
y_train_agreement.shape

(25000, 83)

In [58]:
base_model_history = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
                                    validation_data=(X_train[16667:], y_train_tanh[16667:]),
                                    batch_size=1, epochs=1)

Train on 16667 samples, validate on 8333 samples
Epoch 1/1
16667/16667 [==============================] - 43s 3ms/step - loss: 0.4185 - acc: 0.6714 - val_loss: 0.4260 - val_acc: 0.7540


In [59]:
combined_model_history = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], 
                                            validation_data=([X_train[16667:], y_train_agreement[16667:]], y_train_original[16667:]),
                                            batch_size=1, epochs=1)

Train on 16667 samples, validate on 8333 samples
Epoch 1/1
16667/16667 [==============================] - 124s 7ms/step - loss: 0.7543 - acc: 0.3197 - val_loss: 0.7294 - val_acc: 0.3109


In [60]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

25000/25000 [==============================] - 2s 89us/step


[0.1549614046549797, 0.79296]

In [61]:
predict = combined_model.predict([X_test, y_test_agreement])

In [62]:
predict = predict.flatten()
predict.shape

(25000,)

In [63]:
predict_class = (predict > 0.5).astype('int32')
predict_class

array([1, 1, 0, ..., 0, 0, 0])

In [64]:
np.sum(y_test_original == predict_class)/len(predict_class)

0.79296

In [65]:
np.sum(y_test_original==0)

12500

In [66]:
score = combined_model.evaluate([X_train, y_train_agreement], y_train_original)
score

25000/25000 [==============================] - 2s 90us/step


[0.6920640222930908, 0.33228]

In [14]:
# independent weight here
# independent_weight_base_model_history
# independent_weight_combined_model_history

In [15]:
# var name
# shared_weight_base_model_history
# shared_weight_combined_model_history

In [16]:
net_weight = base_model.get_weights()

In [17]:
len(net_weight)

2

In [18]:
weight = net_weight[0].flatten()

In [19]:
net_weight[1].shape

(1,)

In [20]:
words = cv.get_feature_names()

In [21]:
indices = np.argsort(weight)

In [22]:
for i in indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

worst 	 -2.925
incoherent 	 -2.741
dull 	 -2.722
mst3k 	 -2.567
fails 	 -2.489
waste 	 -2.471
behave 	 -2.466
unwatchable 	 -2.460
poorly 	 -2.448
unfunny 	 -2.359


In [23]:
pos_indices = indices[::-1]
for i in pos_indices[:10]:
    print('%s \t %.3f' %(words[i], weight[i]))

rare 	 2.948
refreshing 	 2.938
excellent 	 2.669
glad 	 2.406
captures 	 2.381
unforgettable 	 2.350
strength 	 2.313
touching 	 2.281
complaint 	 2.270
flawless 	 2.255


In [24]:
combined_net_weight = combined_model.get_weights()

In [25]:
len(combined_net_weight)

87

In [26]:
ht_weight = []
for i in range(len(combined_net_weight)):
    if i>1 and i < 85:
        print(combined_net_weight[i].flatten())
        ht_weight.append(combined_net_weight[i].flatten())
    else:
        print(i, combined_net_weight[i].shape)
ht_weight = np.asarray(ht_weight)

0 (3641, 1)
1 (1,)
[3.627777]
[3.8900218]
[4.153823]
[4.5703483]
[1.4888272]
[1.1577784]
[4.179051]
[3.9351318]
[3.7010958]
[3.7599983]
[3.0573008]
[3.7975197]
[4.0738215]
[4.2703304]
[3.3386655]
[2.47348]
[2.8444138]
[2.2478364]
[2.773281]
[3.8707283]
[3.868085]
[3.770223]
[3.7256012]
[3.7170255]
[3.8314774]
[4.361408]
[4.174931]
[4.292776]
[2.7837505]
[2.9280767]
[2.880346]
[4.1962867]
[3.1430523]
[3.478287]
[3.2203453]
[3.5235784]
[2.4974473]
[3.1943052]
[2.7004623]
[2.7403023]
[2.6789212]
[4.0552034]
[3.5390275]
[4.463016]
[4.190392]
[3.9007244]
[3.863277]
[3.561584]
[2.7166073]
[4.000068]
[3.4171407]
[3.1526122]
[3.6230867]
[3.6077058]
[3.7694995]
[2.9206834]
[2.1293828]
[4.48624]
[3.8510976]
[2.9109056]
[3.4334617]
[3.208707]
[3.3748553]
[3.6008465]
[3.6978815]
[4.1673093]
[3.6438437]
[2.9172628]
[3.9650507]
[3.1554806]
[3.3657262]
[3.3937044]
[4.0483174]
[4.424462]
[3.6251955]
[3.2002373]
[4.7132535]
[2.5550203]
[3.83853]
[3.3442917]
[1.7070978]
[3.7956536]
[4.2303305]
85 (83, 1

In [27]:
sigmoid_weight = combined_net_weight[85].flatten()

In [28]:
ht_weight = ht_weight.flatten()

In [29]:
human_term_indices = np.argsort(ht_weight)

print('human terms \t dense_relu weight \t sigmoid weight \t\t pos/neg?')
for i in human_term_indices:
    print('%s \t %.3f \t %.3f \t\t %.1f' %(word_list[i], ht_weight[i], sigmoid_weight[i], connotation[word_list[i]]))

human terms 	 dense_relu weight 	 sigmoid weight 		 pos/neg?
6/10 	 1.158 	 0.288 		 1.0
5/10 	 1.489 	 0.637 		 1.0
wonderfully 	 1.707 	 0.903 		 1.0
perfectly 	 2.129 	 1.267 		 1.0
beautifully 	 2.248 	 1.447 		 1.0
badly 	 2.473 	 -1.309 		 0.0
fun 	 2.497 	 1.625 		 1.0
wasted 	 2.555 	 -1.624 		 0.0
great 	 2.679 	 1.746 		 1.0
funniest 	 2.700 	 1.779 		 1.0
loved 	 2.717 	 1.729 		 1.0
gem 	 2.740 	 1.636 		 1.0
best 	 2.773 	 1.582 		 1.0
enjoyable 	 2.784 	 1.942 		 1.0
beautiful 	 2.844 	 1.966 		 1.0
excellent 	 2.880 	 1.972 		 1.0
poorly 	 2.911 	 -1.919 		 0.0
solid 	 2.917 	 1.983 		 1.0
perfect 	 2.921 	 1.878 		 1.0
enjoyed 	 2.928 	 1.964 		 1.0
amazing 	 3.057 	 1.910 		 1.0
fantastic 	 3.143 	 2.006 		 1.0
mst3k 	 3.153 	 -2.150 		 0.0
subtle 	 3.155 	 2.149 		 1.0
funny 	 3.194 	 -2.109 		 0.0
unfunny 	 3.200 	 -2.404 		 0.0
rare 	 3.209 	 2.297 		 1.0
favorite 	 3.220 	 2.317 		 1.0
bad 	 3.339 	 -2.237 		 0.0
wonderful 	 3.344 	 2.164 		 1.0
superb 	 3.366 	 2.

In [30]:
len(connotation)

83

In [20]:
# Print report on the word transparency
index = [9, 19]
def report():
    for i in index:
        print()
        bm = base_model.predict(X_test[i])
        
        cm = combined_model.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
#         document_output = 'multiply'
#         document_predict = Model(inputs=combined_model.input,
#                                      outputs=combined_model.get_layer(document_output).output)
#         doc_output = document_predict.predict([np.reshape(data[i], (1,5)), 
#                                       ht_1_input[i], 
#                                       ht_2_input[i], 
#                                       ht_3_input[i], 
#                                       ht_4_input[i]])
        
        layer_name = 'concatenate'
        concat_after_relu = Model(inputs=combined_model.input,
                                     outputs=combined_model.get_layer(layer_name).output)
        concat_output = concat_after_relu.predict([X_test[i], 
                                np.reshape(y_test_agreement[i], (1,y_test_agreement.shape[1]))])
        
        print(X_test_original[i], '\n\n actual label : ', y_test_original[i], '\n predict from base model : ', bm.flatten(), '\n predict label : ', cm.flatten())
    
        for i,output in enumerate(concat_output.flatten()):
            if output != 0:
                print(word_list[i], output)

In [21]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [22]:
report()


hilarious, clean, light-hearted, and quote-worthy. what else can you ask for in a film? this is my all-time, number one favorite movie. ever since i was a little girl, i have dreamed of owning a blue van with flame and an observation bubble.the cliché character in ridiculous situation are what make this film such great fun. the wonderful comedic chemistry between stephen furst (harold) and andy tennant (melio) make up most of my favorite part of the movie. and who did not love the hopeless awkwardness of flynch? do not forget the airport antic of leon's crony, dressed up as hari krishna: dancing, chanting and playing the tambourine--unbeatable! the clue are genius, the location are classic, and the plot is timeless.a word to the wise, if you did not watch this film when you were little, it probably will not win a place in your heart today. but nevertheless give it a chance, you may find that "it does not matter what you say, it does not matter what you do, you have gotta play." 

 act

In [14]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)

25000/25000 [==============================] - 2s 88us/step


In [15]:
score

[0.16272140228033066, 0.78232]

### Let's see the weight when the trainable is false

In [36]:
def build_combined_model():
    # build the combined model
    # Combined model
    human_terms_len = len(word_list)

    base_model = build_base_model(X_train.shape[1])

    combined_input_layer = Input(shape=(X_train.shape[1],))

    # build the hard coded weight for human terms
    ht_input_layer = Input(shape=(human_terms_len,))

    split = Lambda( lambda x: tf.split(x,num_or_size_splits=human_terms_len,axis=1))(ht_input_layer)

    # get the document prediction
    label_layer = base_model(combined_input_layer)

    # stack the multiply layer
    dense_layer = []
    for i in range(human_terms_len):
        dense_layer.append(Dense(1, activation='relu',use_bias=False, kernel_initializer='ones', trainable=True)(Multiply()([split[i], label_layer])))

    # concat all the result   
    concat = Lambda( lambda x: tf.concat(x, axis=1), name='concatenate')(dense_layer)

    # pass it to sigmoid layer
    output_layer = Dense(1, activation='sigmoid')(concat)

    combined_model = Model(inputs=[combined_input_layer, ht_input_layer], outputs=output_layer)
    combined_model.summary()
    
    return base_model, combined_model

In [37]:
# base_model, combined_model = build_combined_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3641)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 3642      
Total params: 3,642
Trainable params: 3,642
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 83)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 3641)         0                                            
_________________________________________

In [38]:
# base_model.compile(loss='mse',
#                   optimizer='Adagrad',
#                   metrics=['acc'])

# combined_model.compile(loss='mse',
#                       optimizer='Adagrad',
#                       metrics=['acc'])

# base_model_history_train_false = base_model.fit(X_train[:16667], y_train_tanh[:16667], 
#                                     validation_data=(X_train[16667:], y_train_tanh[16667:]),
#                                     batch_size=1, epochs=50)

# combined_model_history_train_false = combined_model.fit([X_train[:16667],y_train_agreement[:16667]], y_train_original[:16667], batch_size=1, epochs=50)

In [41]:
# base_model_history_trainfalse = base_model_history
# combined_model_history_trainfalse = combined_model_history

In [42]:
net_weights = combined_model.get_weights()

In [43]:
# Evaluate model

### Generate color weighted

In [19]:
import re

class ColoredWeightedDoc(object):
    def __init__(self, doc, human_terms, ht_weights, token_pattern=r"(?u)\b\w\w+\b", binary = False):
        self.doc = doc
        self.human_terms = human_terms
        self.ht_weights = ht_weights
        self.binary = binary
        self.tokenizer = re.compile(token_pattern)
#         self.abs_ranges = np.linspace(0, max([abs(coefs.min()), abs(coefs.max())]), 8)
    def _repr_html_(self):
        html_rep = ""
        tokens = self.doc.split(" ") 
        if self.binary:
            seen_tokens = set()       
        for token in tokens:
            vocab_tokens = self.tokenizer.findall(token.lower())
            if len(vocab_tokens) > 0:
                vocab_token = vocab_tokens[0]
                try:
                    vocab_index = self.human_terms.index(vocab_token)
                    
                    if not self.binary or vocab_index not in seen_tokens:
                        
                        if self.ht_weights[vocab_index] == 0: # human-terms which has been washed out (opposing)
                            html_rep = html_rep + "<font size = 2, color=lightgreen> " + token + " </font>"
                        
                        elif self.ht_weights[vocab_index] != 0: # human-terms transparency
                            html_rep = html_rep + "<font size = 3, color=blue> " + token + " </font>"
                        
                        else: # neutral word
                            html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                        
                        if self.binary:    
                            seen_tokens.add(vocab_index)
                    
                    else: # if binary and this is a token we have seen before
                        html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
                except: # this token does not exist in the vocabulary
                    html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
            else:
                html_rep = html_rep + "<font size = 1, color=gainsboro> " + token + " </font>"
        return html_rep

In [20]:
from IPython import display
idx = 1001
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  0 
predict from base model :  [-1.] 
predict label :  [0.]

awful 4.363293
boring 3.6333165
horrible 4.4387865
pathetic 3.9187655
ridiculous 3.9718459
stupid 3.8435516
waste 4.471544


In [21]:
np.where(np.sum(y_test_agreement, axis=1)!=0)[0][1000:1010]

array([1231, 1232, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241],
      dtype=int64)

In [22]:
score = combined_model.evaluate([X_test, y_test_agreement], y_test_original)
score

25000/25000 [==============================] - 2s 60us/step


[0.17322103708982467, 0.8082]

In [23]:
concat_all = concat_after_relu.predict([X_test, y_test_agreement])

In [24]:
concat_all.shape

(25000, 83)

In [25]:
indices = np.where(np.sum(concat_all, axis=1)!=0)
indices = indices[0]
len(indices)

21567

In [26]:
not_zero_predict = combined_model.evaluate([X_test[indices], y_test_agreement[indices]], y_test_original[indices])

21567/21567 [==============================] - 1s 59us/step


In [27]:
not_zero_predict

[0.12119727466684344, 0.8748551026700396]

In [29]:
(X_test.shape[0] - len(indices))/X_test.shape[0]

0.13732

In [37]:
combined_model.metrics_names

['loss', 'acc']

In [30]:
combined_model.save_weights('imdb_human_term_weight.hdf5')

# Get cases

In [118]:
from IPython import display
idx = 7079
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  1 
predict from base model :  [0.9990606] 
predict label :  [1.]

7/10 3.6766682
amazing 3.7045205
beautiful 3.1186619
fun 3.3016078
loved 3.1563363
wonderful 3.3778563


In [119]:
from IPython import display
idx = 11050
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  1 
predict from base model :  [0.9999331] 
predict label :  [1.]

best 3.400367
fascinating 3.7045162
favorite 3.4731314
great 3.0944543
subtle 3.3368397
wonderful 3.3808064
wonderfully 1.2528424


In [125]:
from IPython import display
idx = 21625
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output, connotation[word_list[i]])


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  1 
predict from base model :  [-1.] 
predict label :  [8.4540716e-36]

avoid 4.1246505 0
bad 3.54954 0
boring 3.6333165 0
dull 4.256631 0
funny 3.3598797 0
pointless 4.2227383 0
worst 4.2214837 0


In [124]:
from IPython import display
idx = 2494
# idx = 9002
bm = base_model.predict(X_test[idx])
        
cm = combined_model.predict([X_test[idx], 
                             np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
layer_name = 'concatenate'
concat_after_relu = Model(inputs=combined_model.input,
                          outputs=combined_model.get_layer(layer_name).output)
concat_output = concat_after_relu.predict([X_test[idx], np.reshape(y_test_agreement[idx], (1,y_test_agreement.shape[1]))])
        
print('actual label : ', y_test_original[idx], '\npredict from base model : ', bm.flatten(), '\npredict label : ', cm.flatten())
print()
for i,output in enumerate(concat_output.flatten()):
    if output != 0:
        print(word_list[i], output)


ht_weight = concat_output.flatten()
display.display(ColoredWeightedDoc(X_test_original[idx], word_list, ht_weight, binary = False))

actual label :  0 
predict from base model :  [0.99637777] 
predict label :  [1.]

8/10 3.6608827
amazing 3.6945724
beautiful 3.1102872
best 3.3882768
fun 3.2927418
solid 3.2650845
wonderful 3.3687856


In [69]:
y_test_agreement[14689]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [83]:
positive = np.sum(y_test_agreement==1, axis=1)

In [84]:
negative = np.sum(y_test_agreement==-1, axis=1)

In [85]:
np.argmax(positive)

21728

In [36]:
np.max(negative)

17

In [43]:
X_train_original[np.argmax(negative)]

'the movie has a distinct (albeit utish and rough) humanity for all its borderline depravity - the zippy/lyrical score point up the comic side of their misadventure, and even when they are at their most thuggish (like terrorizing the woman on the train), a semi-pitiful vulnerability lurk never far away (dewaere suck on her east like a baby). blier cut away from the scene where depardieu may be about to rape dewaere, so we are never sure how explicitly to read the manifestly homoerotic aspect of their relationship - either way, that incident is the start of their relative humanization (so the movie could certainly be read as pro-gay, although it could likely be read as pro-anything you want). the movie has many objectionable scene and point of sexual politic and is probably best taken as a general cartoon on the foible of both sex, making a mockery of the whole notion of sensitivity and honesty, and hitting numerous point of possible profundity on the basis that if you fire off enough s

In [59]:
np.argmax(negative)

4378

In [60]:
np.argmax(positive)

21728

In [87]:
positive = positive.reshape(25000, 1)
negative = negative.reshape(25000, 1)

stack = np.hstack([positive, negative])
stack.shape

(25000, 2)

In [90]:
new = np.absolute(stack[:,0] - stack[:,1])

In [122]:
ind = []
for i in range(new.shape[0]):
    if new[i] <= 1 and (stack[i,0] >6  or stack[i,1] >6):
        ind.append(i)

In [123]:
ind

[2494, 7079, 11050, 12559, 20031, 21625, 21874, 23852]

In [111]:
stack_balance = []
for i in ind:
    stack_balance.append(stack[i])

In [112]:
stack_balance

[]

In [102]:
np.argmax(stack_balance)

510

In [103]:
stack_balance[510]

array([4, 4])

In [80]:
np.argmax(np.sum(stack, axis=1))

4378